In [ ]:
# 📦 Criação e Configuração do Banco de Dados

Este notebook conecta ao banco PostgreSQL criado no Docker e executa os comandos SQL necessários para estruturar o banco de dados. Além de criar as tabelas, também configura índices para otimização de consultas e insere dados iniciais para teste.

## Objetivos:
- Estabelecer conexão segura com o PostgreSQL
- Criar esquema de tabelas para dados brutos e predições
- Configurar índices para otimização de performance
- Inserir dados de exemplo para validação


In [ ]:
#conexão com o banco de dados (com tratamentos de erros)

import psycopg2
import os
from dotenv import load_dotenv
from psycopg2 import OperationalError

# Carregar variáveis de ambiente
load_dotenv()

# Parâmetros de conexão
db_params = {
    "host": os.getenv("DB_HOST", "localhost"),
    "port": os.getenv("DB_PORT", "5433"),
    "database": os.getenv("DB_NAME", "weather"),
    "user": os.getenv("DB_USER", "postgres"),
    "password": os.getenv("DB_PASSWORD", "postgres")
}

# Função para conectar com tratamento de erros
def connect_to_db(params):
    try:
        conn = psycopg2.connect(**params)
        print("✅ Conexão estabelecida com sucesso!")
        return conn
    except OperationalError as e:
        print(f"❌ Erro ao conectar ao banco de dados: {e}")
        return None

# Estabelecer conexão
conn = connect_to_db(db_params)

if conn:
    cursor = conn.cursor()
else:
    raise Exception("Não foi possível estabelecer conexão com o banco de dados.")

In [ ]:
# criar tabela raw_data 

# Definição da tabela para dados brutos climáticos
create_raw_data_table = """
CREATE TABLE IF NOT EXISTS raw_data (
    id SERIAL PRIMARY KEY,
    city VARCHAR(50) NOT NULL,
    temperature REAL NOT NULL,
    humidity REAL NOT NULL,
    timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""

try:
    cursor.execute(create_raw_data_table)
    conn.commit()
    print("✅ Tabela 'raw_data' criada com sucesso!")
except Exception as e:
    conn.rollback()
    print(f"❌ Erro ao criar tabela 'raw_data': {e}")

In [ ]:
#criar tabela model_predictions 

# Definição da tabela para armazenar predições do modelo
create_predictions_table = """
CREATE TABLE IF NOT EXISTS model_predictions (
    id SERIAL PRIMARY KEY,
    humidity REAL NOT NULL,
    predicted_temperature REAL NOT NULL,
    actual_temperature REAL,
    prediction_error REAL,
    model_version VARCHAR(50),
    timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""

try:
    cursor.execute(create_predictions_table)
    conn.commit()
    print("✅ Tabela 'model_predictions' criada com sucesso!")
except Exception as e:
    conn.rollback()
    print(f"❌ Erro ao criar tabela 'model_predictions': {e}")

In [ ]:
# criação de indices para otimização 

# Criar índices para otimizar consultas frequentes
create_indices = [
    "CREATE INDEX IF NOT EXISTS idx_raw_data_city ON raw_data(city);",
    "CREATE INDEX IF NOT EXISTS idx_raw_data_timestamp ON raw_data(timestamp);",
    "CREATE INDEX IF NOT EXISTS idx_predictions_timestamp ON model_predictions(timestamp);"
]

for index_query in create_indices:
    try:
        cursor.execute(index_query)
        conn.commit()
        print(f"✅ Índice criado: {index_query}")
    except Exception as e:
        conn.rollback()
        print(f"❌ Erro ao criar índice: {e}")

In [ ]:
# inserção de dados iniciais para teste 

from datetime import datetime, timedelta
import random

# Gerar dados fictícios para cinco cidades
cities = ['São Paulo', 'Rio de Janeiro', 'Brasília', 'Curitiba', 'Recife']
base_time = datetime.now() - timedelta(days=30)  # Dados dos últimos 30 dias

# Preparar dados para inserção
sample_data = []
for city in cities:
    # Gerar 10 registros por cidade
    for i in range(10):
        # Gerar timestamp com intervalos de 3 horas
        timestamp = base_time + timedelta(hours=3*i)
        # Temperatura em Celsius (variando entre 15-35)
        temperature = round(random.uniform(15, 35), 1)
        # Umidade em % (variando entre 30-90)
        humidity = round(random.uniform(30, 90), 1)
        sample_data.append((city, temperature, humidity, timestamp))

# SQL para inserção
insert_query = """
INSERT INTO raw_data (city, temperature, humidity, timestamp)
VALUES (%s, %s, %s, %s);
"""

try:
    cursor.executemany(insert_query, sample_data)
    conn.commit()
    print(f"✅ {len(sample_data)} registros de exemplo inseridos com sucesso!")
except Exception as e:
    conn.rollback()
    print(f"❌ Erro ao inserir dados de exemplo: {e}")

In [ ]:
# encerramento e validação 

# Verificar os dados inseridos
cursor.execute("SELECT COUNT(*) FROM raw_data")
count = cursor.fetchone()[0]
print(f"Total de registros na tabela raw_data: {count}")

# Verificar estrutura da tabela
cursor.execute("""
SELECT column_name, data_type 
FROM information_schema.columns 
WHERE table_name = 'raw_data'
""")
columns = cursor.fetchall()
print("\nEstrutura da tabela raw_data:")
for col in columns:
    print(f"- {col[0]}: {col[1]}")

# Fechar conexão
cursor.close()
conn.close()
print("\n✅ Processo concluído! Banco de dados configurado com sucesso.")